In [9]:
import numpy as np
from scipy.stats import norm

# Given data
T = 61/ 252  # Convert days to years
r = 0.0485  # Risk-free rate

# Data for XLE, XLF, XLY
etfs = {
    "XLE": {"S0": 84.36, "K": 84, "sigma": 0.317964, "D": 0.85963},
    "XLF": {"S0": 33.70, "K": 34, "sigma": 0.227176, "D": 0.20962},
    "XLY": {"S0": 133.91, "K": 134, "sigma": 0.290478, "D": 0.36581},
}

# Function to compute Black-Scholes price for a call option with dividends
def black_scholes_dividend(S0, K, r, T, sigma, D):
    q = D / S0  # Approximate continuous dividend yield
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Compute option prices
option_prices = {etf: black_scholes_dividend(data["S0"], data["K"], r, T, data["sigma"], data["D"]) for etf, data in etfs.items()}

# Print results
for etf, price in option_prices.items():
    print(f"Option Price for {etf}: ${price:.2f}")


Option Price for XLE: $5.80
Option Price for XLF: $1.52
Option Price for XLY: $8.30


In [10]:
import numpy as np

def binomial_call_price(S0, K, r, T, sigma, q, N):
    """
    Price a European call option using a CRR binomial tree with continuous dividend yield.

    Parameters:
      S0    : Current stock price
      K     : Strike price
      r     : Continuously compounded risk-free rate
      T     : Time to expiration in years
      sigma : Volatility of the underlying
      q     : Continuous dividend yield
      N     : Number of timesteps

    Returns:
      call_price : Option price computed via the binomial tree
    """
    dt = T / N
    # Up and Down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = np.exp(-sigma * np.sqrt(dt))

    # Risk-neutral probability (adjusted for dividend yield)
    p = (np.exp((r - q) * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    asset_prices = np.array([S0 * (u ** j) * (d ** (N - j)) for j in range(N + 1)])
    # Option payoffs at maturity: max(S-K,0)
    option_values = np.maximum(asset_prices - K, 0)

    # Backward induction
    discount = np.exp(-r * dt)
    for i in range(N, 0, -1):
        option_values = discount * (p * option_values[1:i+1] + (1 - p) * option_values[0:i])

    return option_values[0]

# Inputs for the three ETFs
# Using trading days: T = 61/252 years (61 trading days from Dec 16, 2022 to Mar 17, 2023)
T = 61 / 252.0
r = 0.0485
N_tree = 1000  # Number of timesteps

# Data for each ETF
etfs = {
    "XLE": {
        "S0": 84.36,
        "K": 84.0,
        "sigma": 0.317964,
        "dividend": 0.85963  # dividend amount
    },
    "XLF": {
        "S0": 33.70,
        "K": 34.0,
        "sigma": 0.227176,
        "dividend": 0.20962
    },
    "XLY": {
        "S0": 133.91,
        "K": 134.0,
        "sigma": 0.290478,
        "dividend": 0.36581
    }
}

# For each ETF, we compute the approximate continuous dividend yield as dividend / S0.
for etf in etfs:
    S0 = etfs[etf]["S0"]
    etfs[etf]["q"] = etfs[etf]["dividend"] / S0

# Price the options for each ETF
option_prices = {}
for etf, params in etfs.items():
    price = binomial_call_price(
        S0=params["S0"],
        K=params["K"],
        r=r,
        T=T,
        sigma=params["sigma"],
        q=params["q"],
        N=N_tree
    )
    option_prices[etf] = price

# Print the results
print("91-day call option prices using the binomial model (1,000 timesteps):")
for etf, price in option_prices.items():
    print(f"  {etf}: ${price:.2f}")


91-day call option prices using the binomial model (1,000 timesteps):
  XLE: $5.80
  XLF: $1.52
  XLY: $8.30
